In [2]:
#导入库
from keras.layers import Input, Dense, Flatten, Dropout, Activation 
from keras.layers.normalization import BatchNormalization
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import TensorBoard
from keras.preprocessing import image
from keras.models import load_model
from keras.models import Model
import matplotlib.pyplot as plt
import glob, os, cv2, random,time
import numpy as np
from keras.models import Sequential
from keras.layers import Conv2D,Flatten,MaxPooling2D,Dense 
from keras.optimizers import SGD
from keras.applications.vgg16 import VGG16

# 数据预处理
data_path: 数据集路径
train, test:处理后的训练集数据、测试集数据

In [3]:
def processing_data(data_path):
    #生成训练集
    train_data = ImageDataGenerator(
            # 对图片的每个像素值均乘上这个放缩因子，把像素值放缩到0和1之间有利于模型的收敛
            rescale=1. / 225,  
            # 浮点数，剪切强度（逆时针方向的剪切变换角度）
            shear_range=0.1,  
            # 随机缩放的幅度，若为浮点数，则相当于[lower,upper] = [1 - zoom_range, 1+zoom_range]
            zoom_range=0.1,
            # 浮点数，图片宽度的某个比例，数据提升时图片水平偏移的幅度
            width_shift_range=0.1,
            # 浮点数，图片高度的某个比例，数据提升时图片竖直偏移的幅度
            height_shift_range=0.1,
            # 布尔值，进行随机水平翻转
            horizontal_flip=True,
            # 布尔值，进行随机竖直翻转
            vertical_flip=True,
            # 在 0 和 1 之间浮动。用作验证集的训练数据的比例
            validation_split=0.1  
    )
    # 生成测试集
    validation_data = ImageDataGenerator(
            rescale=1. / 255,
            validation_split=0.1)
    
    #以文件夹路径为参数,生成经过归一化后的数据,在一个无限循环中无限产生batch数据
    train_generator = train_data.flow_from_directory(
            # 提供的路径下面需要有子目录
            data_path, 
            # 整数元组 (height, width)，默认：(256, 256)。 所有的图像将被调整到的尺寸。
            target_size=(150, 150),
            # 一批数据的大小
            batch_size=256,
            # "categorical", "binary", "sparse", "input" 或 None 之一。
            # 默认："categorical",返回one-hot 编码标签。
            class_mode='categorical',
            # 数据子集 ("training" 或 "validation")
            subset='training', 
            seed=0)
    validation_generator = validation_data.flow_from_directory(
            data_path,
            target_size=(150, 150),
            batch_size=256,
            class_mode='categorical',
            subset='validation',
            seed=0)

    return train_generator, validation_generator

# 模型的建立
本次实验采用Vgg16模型

In [26]:
def model(train_generator, validation_generator, save_model_path):
    '''
    weights='imagenet' 使用ImageNet上预训练的模型，用ImageNet的参数初始化模型的参数
    include_top = False 是否包含最上层的全连接层 表明迁移除顶层以外的其余网络结构到自己的模型中
    input_shape 输入进来的数据是150*150 3通道
    通过.add()方法一个个的将layer加入模型中
    '''
    vgg16_model = VGG16(weights='imagenet',include_top=False, input_shape=(150,150,3))
    top_model = Sequential()
    # Flatten层用来将输入“压平”，即把多维的输入一维化，常用在从卷积层到全连接层的过渡
    top_model.add(Flatten(input_shape=vgg16_model.output_shape[1:]))
    # dense 全连接 activation: 激活函数
    top_model.add(Dense(256,activation='relu'))
    top_model.add(Dropout(0.5))
    top_model.add(Dense(4,activation='softmax'))

    model = Sequential()
    model.add(vgg16_model)
    model.add(top_model)
    # 配置模型学习过程, 采用 compile 函数: https://keras.io/models/model/#compile
    model.compile(
             # 优化器, 主要有Adam、sgd、rmsprop等方式。
            # SGD 每次更新时对每个样本进行梯度更新, 一次只进行一次更新，就没有冗余，而且比较快，并且可以新增样本
            # lr: float >= 0. 学习率, momentum: float >= 0. 参数，用于加速 SGD 在相关方向上前进，并抑制震荡
            optimizer=SGD(lr=1e-3,momentum=0.9),
            # 损失函数,多分类采用 categorical_crossentropy
            loss='categorical_crossentropy',
            # 评价函数 训练和测试期间的模型评估标准 除了损失函数值之外的特定指标, 分类问题一般都是准确率
            metrics=['accuracy'])
    # 训练
    model.fit_generator(
            # 一个生成器或 Sequence 对象的实例
            generator=train_generator,
            # epochs: 整数，数据的迭代总轮数。
            epochs=3,
            # 一个epoch包含的步数,通常应该等于你的数据集的样本数量除以批量大小。
            steps_per_epoch=9373 //256,
            # 验证集
            validation_data=validation_generator,
             # 在验证集上,一个epoch包含的步数,通常应该等于你的数据集的样本数量除以批量大小。
            validation_steps=1039// 256,
            )
    model.save(save_model_path)
    return model
# 评估模型
def evaluate_mode(validation_generator, save_model_path):
     # 加载模型
    model = load_model('D:/Anaconda/Anaconda3/envs/tensorflow/Lib/site-packages/tensorflow_core/python/keras/api/keras/models/vgg16.h5')
    # 获取验证集的 loss 和 accuracy
    loss, accuracy = model.evaluate_generator(validation_generator)
    print("\nLoss: %.2f, Accuracy: %.2f%%" % (loss, accuracy * 100))

In [5]:
def main():
    """
    深度学习模型训练流程,包含数据处理、创建模型、训练模型、模型保存、评价模型等。
    如果对训练出来的模型不满意,你可以通过调整模型的参数等方法重新训练模型,直至训练出你满意的模型。
    如果你对自己训练出来的模型非常满意,则可以提交作业!
    :return:
    """
    data_path = "train/"  # 数据集路径
    save_model_path = 'D:/Anaconda/Anaconda3/envs/tensorflow/Lib/site-packages/tensorflow_core/python/keras/api/keras/models/vgg16.h5'  # 保存模型路径和名称
    # 获取数据
    train_generator, validation_generator = processing_data(data_path)
    # 创建、训练和保存模型
    model(train_generator, validation_generator, save_model_path)
    # 评估模型
    evaluate_mode(validation_generator, save_model_path)

if __name__ == '__main__':
    main()

Found 9373 images belonging to 4 classes.
Found 1039 images belonging to 4 classes.
Epoch 1/3
36/36 [==============================] - 3265s 91s/step - loss: 0.9083 - accuracy: 0.6334 - val_loss: 0.5854 - val_accuracy: 0.7988
Epoch 2/3
36/36 [==============================] - 3288s 91s/step - loss: 0.4608 - accuracy: 0.8322 - val_loss: 0.6355 - val_accuracy: 0.7918
Epoch 3/3
36/36 [==============================] - 3258s 91s/step - loss: 0.3367 - accuracy: 0.8796 - val_loss: 0.4528 - val_accuracy: 0.8582

Loss: 0.46, Accuracy: 84.31%


# 模型预测

In [88]:
def predict(img_path):
    global model
    # 把图片转换成为numpy数组
    img = img = image.load_img(img_path, target_size=(150, 150))
    img = image.img_to_array(img)

    # expand_dims的作用是把img.shape转换成(1, img.shape[0], img.shape[1], img.shape[2])
    x = np.expand_dims(img, axis=0)
    # 模型预测
    y =model.predict(x)

    # 获取labels
    labels = {0: '厨余垃圾', 1: '可回收物', 2: '其他垃圾', 3: '有害垃圾'}
    
    # -------------------------------------------------------------------------
    predict = labels[np.argmax(y)]
    

    # 返回图片的类别
    return predict
img_path = 'testImg/img_PE塑料袋_2.jpg'
frame = cv2.imread(img_path)
# 定义字体
font = cv2.FONT_HERSHEY_SIMPLEX
cv2.putText(frame,predict(img_path), (10, 140), font, 3, (255, 0, 0), 2, cv2.LINE_AA)
cv2.imwrite('results/pred.jpg', frame)
cv2.imshow('img', frame)
cv2.waitKey(0)

AttributeError: 'function' object has no attribute 'predict'

In [78]:
# coding=utf-8
import tkinter as tk
from tkinter import *
from tkinter import ttk
from PIL import Image, ImageTk
 
# 创建窗口 设定大小并命名
window = tk.Tk()
window.title('垃圾分类')
window.geometry('1050x500')
global img_png           # 定义全局变量 图像的
var = tk.StringVar()    # 这时文字变量储存器
 
mainframe = ttk.Frame(window, padding="5 4 12 12")
mainframe.grid(column=0, row=0, sticky=(N, W, E, S))
mainframe.columnconfigure(0, weight=1)
mainframe.rowconfigure(0, weight=1)
 
def openImg():
    global img_png
    var.set('已打开')
    Img = Image.open('test/img_奥利奥饼干_19.jpg')
    img_png = ImageTk.PhotoImage(Img)
    label_Img2 = tk.Label(image=img_png).grid(column=2, row=2, sticky=W)
 
num = 1
def change():       #更新图片操作
    global num
    var.set('已预测')
    num=num+1
    if num%3==0:
        url1="results/pred.jpg"
        pil_image = Image.open(url1)
        img= ImageTk.PhotoImage(pil_image)
        label_img.configure(image = img)
    window.update_idletasks()   #更新图片，必须update
 
# row = 1
# epochs = StringVar()
# ttk.Label(mainframe, text="epochs:").grid(column=1, row=1, sticky=W)
# addr_entry = ttk.Entry(mainframe, width=7, textvariable=epochs)
# addr_entry.grid(column=2, row=1, sticky=(W, E))
#
# ttk.Button(mainframe, text="Train").grid(column=4, row=1, sticky=W)
 
# row = 2
ttk.Button(mainframe, text="打开", command=openImg).grid(column=1, row=2, sticky=W)
ttk.Button(mainframe, text="预测", command=change).grid(column=3, row=2, sticky=W)
 
# row = 3 创建文本窗口，显示当前操作状态
ttk.Label(mainframe, text="状态").grid(column=1, row=3, sticky=W)
ttk.Label(mainframe, textvariable=var).grid(column=3, row=3, sticky=W)
 
url = "testImg/logo.jpg"
pil_image = Image.open(url)
img= ImageTk.PhotoImage(pil_image)
label_img = ttk.Label(window, image = img ,compound=CENTER)
label_img.grid(column=0, row=2, sticky=W)
 
# 运行整体窗口
window.mainloop()

TclError: image "pyimage2" doesn't exist